In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
import pyautogui
from selenium.webdriver import ActionChains
from pynput.keyboard import Key, Controller
import pandas as pd
import requests
from selenium.webdriver.common.by import By
from pathlib import Path
def find_data():
    driver = webdriver.Chrome("D:\東海研究所\論文\chromedriver.exe")
    driver.get("https://www.google.com.tw/maps/search/%E5%8F%B0%E4%B8%AD%E7%BE%8E%E9%A3%9F/@24.1623009,120.6227675,14z/data=!3m1!4b1?hl=zh-TW")
    for h in range(1, 51):
        element = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a')
        actionChains = ActionChains(driver)
        actionChains.context_click(element).send_keys(Keys.ARROW_DOWN).perform()
        time.sleep(3)
        keyboard = Controller()
        keyboard.press(Key.esc)
        keyboard.release(Key.esc)
        time.sleep(2)
        for j in range(15):
                time.sleep(0.5)
                pyautogui.press('pgdn')

        soup = BeautifulSoup(driver.page_source)
        href1 = soup.find_all('a', {'class': 'a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd'})
        href_link_list = []
        title_list = []
        for i in href1:
                href_link = i.get('href')
                title = i.get('aria-label')
                href_link_list.append(href_link)
                title_list.append(title)
        find_star = soup.find_all('span',{'class':'MW4etd'})
        star_list = []
        for r in find_star:
            star_list.append(r.text)
        find_comm = soup.find_all('span',{'class':'UY7F9'})
        comm_list = []
        for kk in find_comm:
            comm_list.append(kk.text.replace('(','').replace(')','').replace(',',''))
        df = pd.DataFrame()
        df['店名'] = title_list
        df['網址'] = href_link_list
        df['星級'] = star_list
        df['評論'] = comm_list
        df.to_csv('data/'+str(h)+'商家網址連結.csv', index=False, encoding='utf-8-sig')
        driver.find_element_by_xpath('//*[@id="ppdPk-Ej1Yeb-LgbsSe-tJiF1e"]').click()
        time.sleep(3)
find_data()

def marge_data():
    excel_dir = Path('D:\東海研究所\論文\data')
    excel_files = excel_dir.glob('*.csv')
    df = pd.DataFrame()
    for xls in excel_files:
        data = pd.read_csv(xls, sep=',' )
        df = df.append(data)
    df.to_csv("商家網址連結.csv", encoding='utf-8-sig', index=False, sep=',')
marge_data()


def select_data():
    df = pd.read_csv('商家網址連結.csv')
    df = df[df["評論"] > 500]
    df.to_csv('ee.csv', encoding='utf-8-sig', index=False)
select_data()


def get_image():
    df = pd.read_csv('ee.csv')
    link = list(df['網址'])
    for i in range(len(link)):
        try:
            driver = webdriver.Chrome("D:\東海研究所\論文\chromedriver.exe")
            driver.get(link[i])
            time.sleep(3)
            driver.find_element_by_xpath(
                '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').click()
            time.sleep(3)
            for j in range(1):
                time.sleep(0.5)
                pyautogui.press('pgdn')
            time.sleep(3)
            driver.find_element(By.XPATH, '//button[@aria-label="餐飲"]').click()
            driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[3]/div[1]/div[1]/div/a').click()
            time.sleep(2)
            for j in range(5):
                time.sleep(0.5)
                pyautogui.press('pgdn')
            time.sleep(3)
            soup = BeautifulSoup(driver.page_source)
            img = soup.find_all('div', {'class': 'mWq4Rd-HiaYvf-MNynB-gevUs'})
            ti = soup.find('div', {'class': 'piCU0 gm2-headline-6'}).text.replace('/', '').replace('|', '')
            img_list = []
            img_list1 = []
            for j in img:
                img_list.append(j.get('style').strip('background-image').replace(': url("', '').replace('");', '').replace(':url(//:0)', ''))
            for i in img_list:
                if len(i)>0:
                    img_list1.append(i)
            for p in range(len(img_list1)):
                img = requests.get(img_list1[p])
                with open("images\\" + str(ti) + str(p + 1) + ".jpg", "wb") as file:  # 開啟資料夾及命名圖片檔
                    file.write(img.content)
                file.close()
            print(img_list1)
            driver.close()
        except:
            print("none")
get_image()